# QEPC Pipeline Smoketest – 2025 Edition

This notebook is the **single source of truth** to confirm the entire QEPC engine is healthy.
Run all cells top → bottom. If you reach the end with a nice prediction table → everything works!

Works even if you are a complete beginner — no errors guaranteed.

In [ ]:
# STEP 0: QEPC project bootstrap (works from any notebook folder)

import sys
from pathlib import Path

def find_project_root() -> Path:
    """
    Walk upwards from the current working directory until we find
    a folder that looks like the QEPC project root
    (has main.py and a qepc/ package).
    """
    here = Path.cwd()
    for parent in [here] + list(here.parents):
        if (parent / "main.py").exists() and (parent / "qepc").exists():
            return parent
    # Fallback: just use cwd
    return here

PROJECT_ROOT = find_project_root()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("QEPC project root:", PROJECT_ROOT)

import qepc_autoload as qa
from qepc_autoload import qepc_step
import pandas as pd

qepc_step("QEPC pipeline smoketest started")


In [ ]:
# STEP 1: Import strengths, lambda engine, simulator

from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation

qepc_step("Imported strengths_v2, lambda_engine, and simulator successfully")


In [ ]:
# STEP 2: Calculate team strengths from Team_Stats

qepc_step("Calculating team strengths from Team_Stats")

strengths_df = calculate_advanced_strengths(
    game_data=None,      # let strengths_v2 load from its default data source
    cutoff_date=None,    # or e.g. "2025-11-01" for backtesting-style runs
    verbose=True,
)

if strengths_df is None or strengths_df.empty:
    raise RuntimeError(
        "[Smoketest] strengths_df is empty. "
        "Check data/raw/Team_Stats.csv or strengths_v2 configuration."
    )

display(strengths_df.head())
print(f"Total teams in strengths_df: {len(strengths_df)}")


In [ ]:
# STEP 3: Create a tiny schedule with 1 test game

qepc_step("Building a tiny test schedule")

if len(strengths_df) < 2:
    raise RuntimeError("[Smoketest] Need at least 2 teams in strengths_df to build a test game.")

home_team = strengths_df.iloc[0]["Team"]
away_team = strengths_df.iloc[1]["Team"]

print("Home team:", home_team)
print("Away team:", away_team)

schedule_df = pd.DataFrame(
    {
        "Home Team": [home_team],
        "Away Team": [away_team],
        # You can add situational columns later:
        # "home_rest_days": [2.0],
        # "away_rest_days": [1.0],
        # "home_b2b": [False],
        # "away_b2b": [False],
    }
)

display(schedule_df)


In [ ]:
# STEP 4: Compute lambda for the test game

qepc_step("Computing lambda for test game")

lambda_df = compute_lambda(
    schedule_df=schedule_df,
    team_stats_df=strengths_df,
    include_situational=False,  # flip to True if you start adding rest/b2b columns
)

if lambda_df is None or lambda_df.empty:
    raise RuntimeError("[Smoketest] lambda_df is empty. Check compute_lambda inputs.")

display(
    lambda_df[["Home Team", "Away Team", "lambda_home", "lambda_away", "vol_home", "vol_away"]]
)


In [ ]:
# STEP 5: Simulate the game

qepc_step("Running QEPC simulation")

sim_results = run_qepc_simulation(
    df=lambda_df,
    num_trials=5000,  # enough for a stable feel but not too slow
)

if sim_results is None or sim_results.empty:
    raise RuntimeError("[Smoketest] Simulation returned empty results.")

display(
    sim_results[
        [
            "Home Team",
            "Away Team",
            "lambda_home",
            "lambda_away",
            "Sim_Home_Score",
            "Sim_Away_Score",
            "Home_Win_Prob",
            "Away_Win_Prob",
            "Expected_Spread",
            "Expected_Score_Total",
        ]
    ]
)
